<a href="https://colab.research.google.com/github/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/blob/main/%E5%B0%88%E9%A1%8C%E5%A0%B1%E5%91%8A_Michael_Jackson_%E5%B0%88%E8%BC%AF%E4%B8%BB%E9%A1%8C%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **專題報告 - Michael Jackson 專輯主題機器人**

### **專案背景**

在現今資訊快速傳播的時代，許多粉絲對偶像歌手的生平與作品充滿興趣，然而相關資訊常分散於不同平台，缺乏一個集中且互動性高的查詢方式。本專題以流行音樂巨星 Michael Jackson 為主題，設計一款專輯介紹與歌曲資料查詢的聊天機器人，結合 LINE 平台提供便捷且趣味的互動體驗，讓使用者隨時透過 LINE 了解 Michael Jackson 各張專輯與歌曲內容，並可查看專輯封面、歌曲年份等資訊。

### 專案目標

- 建立一個以 Michael Jackson 專輯為核心內容的 LINE Bot 聊天機器人。
- 使用者可透過指令或選單查詢各張專輯、曲目、封面、年份等資訊。
- 提供易用的互動式操作介面，增加粉絲參與感與使用樂趣。

### 功能概述

首先，系統具備 專輯選單查詢 功能，使用者可以透過選單介面選擇 Michael Jackson 各張專輯，進一步查看該專輯下的所有歌曲清單。其次，系統支援 歌曲資料呈現，當使用者查詢特定專輯或歌曲時，系統會顯示歌曲名稱、發行年份及相關封面圖片等詳細資訊。同時，透過 圖像訊息傳送 功能，系統可直接在對話中顯示專輯封面圖片，提升整體使用體驗與互動性。系統亦內建 基本指令處理，提供如「幫助」、「回到選單」等基礎操作指令，方便使用者隨時返回主功能表或取得操作說明。最後，為確保穩定運作，系統設有 錯誤處理 機制，能針對可能發生的例外狀況提供適當提示，避免使用過程中出現中斷或異常狀態。

### 系統架構圖

使用者：透過 LINE App 傳送訊息- LINE 平台：透過 webhook 傳送使用者輸入- 應用伺服器（Flask）：接收訊息並處理邏輯- 專輯資料來源：內部預先整理好的專輯與歌曲資料- LINE Bot API：回傳訊息與圖像給使用者

	•	程式語言：Python
	•	框架：Flask（建立 webhook）
	•	部署平台：Google Colab + ngrok
	•	API 使用：LINE Messaging API
	•	資料儲存：內建 JSON 格式專輯資料
	•	圖片處理：透過 URL 連結傳送封面圖片

In [ ]:

!pip install line-bot-sdk==2.4.1 pyngrok google-generativeai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.4.2
    Uninstalling charset-normalizer-3.4.2:
      Successfully uninstalled charset-normalizer-3.4.2
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attem

In [ ]:

import os
import re
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
from pyngrok import ngrok
import google.generativeai as genai
from google.colab import userdata

CHANNEL_ACCESS_TOKEN = userdata.get("LINE_CHANNEL_ACCESS_TOKEN")
CHANNEL_SECRET = userdata.get("LINE_CHANNEL_SECRET")
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")

line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash-latest")
app = Flask(__name__)


In [ ]:

album_data = {
    "Thriller": {"year": 1982, "sales": 70000000, "songs": ["Billie Jean", "Beat It", "Thriller"],
     "cover": "https://upload.wikimedia.org/wikipedia/en/5/55/Michael_Jackson_-_Thriller.png"},
    "Bad": {"year": 1987, "sales": 35000000, "songs": ["Smooth Criminal", "Man in the Mirror", "Bad"],
     "cover": "https://upload.wikimedia.org/wikipedia/en/5/51/Michael_Jackson_-_Bad.png"},
    "Dangerous": {"year": 1991, "sales": 32000000, "songs": ["Black or White", "Remember the Time", "Dangerous"],
     "cover": "https://lh3.googleusercontent.com/acFvHA1OEoI0HBPPG33zidd9n9aG1OTvo7XQQeFjEeQObGv6R3464BvFijHerp3Sit5UeHvQnx6LMoE=w544-h544-l90-rj"},
    "HIStory": {"year": 1995, "sales": 20000000, "songs": ["Scream", "They Don't Care About Us", "Earth Song"],
     "cover": "https://lh3.googleusercontent.com/R7pwf7-lcPvK3dxv8jMkUd4SlbVmrM-nZOFEJqnHGLQFBl4lqj1gyeWSFO5X9HZxgUhTh4KM8n0l7j_k=w544-h544-l90-rj"},
    "Invincible": {"year": 2001, "sales": 6000000, "songs": ["Unbreakable", "You Rock My World", "Threatened"],
     "cover": "https://lh3.googleusercontent.com/L0nwBz3JF8kFa1-1PAOiLC6pB49lsRc3QRCbj1gvU3uCNf4SXnLUGtAFBha1CZ02mhQzMJrcl-rJ0pSG=w544-h544-l90-rj"}
}
vote_counter = {album: 0 for album in album_data}
song_lookup = {song.lower(): album for album, data in album_data.items() for song in data["songs"]}


In [ ]:

def parse_user_intent(user_text):
    prompt = f'''
你是一個 Michael Jackson 專輯聊天機器人，負責幫助使用者查詢專輯資訊、歌曲反查、投票與排行榜查詢。

請判斷使用者意圖，並以以下格式回覆：
[Action]: info / vote / leaderboard / song_search / chat
[Target]: (若有指定專輯或歌曲請列出，否則留空)

使用者輸入: "{user_text}"
'''
    response = model.generate_content([prompt])
    action_match = re.search(r"\[Action\]:\s*(\w+)", response.text)
    target_match = re.search(r"\[Target\]:\s*(.*)", response.text)
    action = action_match.group(1).lower() if action_match else "chat"
    target = target_match.group(1).strip() if target_match else ""
    return action, target


In [ ]:

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


In [ ]:

@handler.add(FollowEvent)
def handle_follow(event):
    welcome_msg = ("👋 歡迎加入 Michael Jackson 專輯智能機器人！\n\n"
                   "您可以輸入歌曲名稱、專輯名稱、menu 查看選單，result 查看排行榜，或直接與我聊天。")
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text=welcome_msg))


In [ ]:

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_text = event.message.text.strip()

    if user_text.lower() == "menu":
        send_carousel(event)
        return

    elif user_text.lower() == "result":
        sorted_votes = sorted(vote_counter.items(), key=lambda x: x[1], reverse=True)
        result_text = "\n".join([f"{i+1}. {name}: {votes}票" for i, (name, votes) in enumerate(sorted_votes)])
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=result_text))
        return

    action, target = parse_user_intent(user_text)

    if action == "info" and target in album_data:
        data = album_data[target]
        songs = "\n".join(f"- {s}" for s in data["songs"])
        reply = f"《{target}》\n年份: {data['year']}\n銷售: {data['sales']//1000000}M\n曲目:\n{songs}"

    elif action == "vote" and target in album_data:
        vote_counter[target] += 1
        trivia_prompt = "請分享一些關於 Michael Jackson 或其音樂專輯的冷知識或小故事，控制在80字內，用繁體中文回答。"
        response = model.generate_content([trivia_prompt])
        trivia = response.text
        reply = (f"✅ 已成功投票給《{target}》！\n目前累計：{vote_counter[target]} 票\n\n🎤 MJ 小知識：{trivia}")

    elif action == "song_search" and target.lower() in song_lookup:
        album_name = song_lookup[target.lower()]
        data = album_data[album_name]
        songs = "\n".join(f"- {s}" for s in data["songs"])
        reply = f"《{album_name}》\n年份: {data['year']}\n銷售: {data['sales']//1000000}M\n曲目:\n{songs}"

    else:
        gemini_response = model.generate_content([user_text])
        reply = gemini_response.text

    line_bot_api.reply_message(event.reply_token, TextSendMessage(text=reply))


In [ ]:

def send_carousel(event):
    columns = []
    for album_name, data in album_data.items():
        column = CarouselColumn(
            thumbnail_image_url=data["cover"],
            title=album_name,
            text=f"銷售：{data['sales']//1000000}M",
            actions=[
                MessageAction(label="專輯資訊", text=f"info {album_name}"),
                MessageAction(label="投票", text=f"vote {album_name}")
            ]
        )
        columns.append(column)

    carousel_template = CarouselTemplate(columns=columns)
    template_message = TemplateSendMessage(alt_text="請選擇你最愛的專輯", template=carousel_template)
    line_bot_api.reply_message(event.reply_token, template_message)


In [ ]:

tunnel = ngrok.connect(5000)
public_url = tunnel.public_url
print("Webhook URL:", public_url + "/callback")
app.run(port=5000)


Webhook URL: https://7556-34-85-137-208.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:58:34] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:58:53] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:58:59] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:59:08] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:59:14] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:59:23] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:59:24] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:59:30] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:59:32] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 05:59:43

# **專案成果與效益**

本專案成功完成一套以 Michael Jackson 專輯為主題的 LINE 聊天機器人系統，使用者可透過簡單的指令操作快速查詢各張專輯及歌曲資訊，並同步顯示封面圖片，提升互動性與使用便利性。系統具備穩定的回應速度與完整的錯誤處理機制，可作為日後擴充其他主題資料機器人的基礎架構。

**系統展示**

In [ ]:
from IPython.display import display, HTML

display(HTML("""
<div style="display:flex;">
    <img src="https://raw.githubusercontent.com/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/c6d467209dd6db3c38827968a8ab592e0318bc85/IMG_5195.PNG" width="300">
    <img src="https://raw.githubusercontent.com/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/c6d467209dd6db3c38827968a8ab592e0318bc85/IMG_5196.PNG" width="300">
    <img src="https://raw.githubusercontent.com/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/c6d467209dd6db3c38827968a8ab592e0318bc85/IMG_5197.PNG" width="300">
    <img src="https://raw.githubusercontent.com/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/c6d467209dd6db3c38827968a8ab592e0318bc85/IMG_5198.PNG" width="300">
</div>
"""))